#  Principal Component Analysis

PCA по своей сути является алгоритмом уменьшения размерности, но он также может быть полезен как инструмент для визуализации, фильтрации шума, извлечения и инженерии признаков и многого другого. После краткого концептуального обсуждения алгоритма PCA мы рассмотрим несколько примеров этих дополнительных приложений.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

## Введение в метод главных компонент

Метод главных компонент (PCA) — это быстрый и гибкий метод для уменьшения размерности данных. Его поведение легче всего визуализировать, рассматривая двумерный набор данных.

In [ ]:
rng = np.random.RandomState(1)
X = np.dot(rng.rand(2, 2), rng.randn(2, 200)).T
plt.scatter(X[:, 0], X[:, 1])
plt.axis('equal');

Видно, что между переменными x и y существует почти линейная зависимость. Это напоминает данные линейной регрессии, которые мы уже рассматривали, но задача здесь немного другая: вместо того чтобы пытаться предсказать значения y на основе значений x, задача заключается в том, чтобы изучить взаимосвязь между значениями x и y.

В методе главных компонент эта взаимосвязь количественно оценивается путем нахождения списка главных осей в данных и использования этих осей для описания набора данных. Воспользуемся PCA из Scikit-Learn.

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca.fit(X)

Обучение модели извлекает некоторые величины из данных, наиболее важными из которых являются компоненты и дисперсия.

In [ ]:
print(pca.components_)

In [ ]:
print(pca.explained_variance_)

Чтобы понять, что означают эти числа, давайте визуализируем их в виде векторов на входных данных, используя компоненты для определения направления вектора и дисперсию для определения длины вектора.

In [ ]:
def draw_vector(v0, v1, ax=None):
    ax = ax or plt.gca()
    arrowprops=dict(arrowstyle='->', linewidth=2,
                    shrinkA=0, shrinkB=0)
    ax.annotate('', v1, v0, arrowprops=arrowprops)

# plot data
plt.scatter(X[:, 0], X[:, 1], alpha=0.2)
for length, vector in zip(pca.explained_variance_, pca.components_):
    v = vector * 3 * np.sqrt(length)
    draw_vector(pca.mean_, pca.mean_ + v)
plt.axis('equal');

Эти векторы представляют собой главные оси данных, и длина каждого вектора указывает на то, насколько "важна" эта ось в описании распределения данных — более точно, это мера дисперсии данных при проекции на эту ось. Проекция каждой точки данных на главные оси является главной компонентой данных.

Преобразование из осей данных в главные оси является *аффинным преобразованием*, что означает, что оно состоит из переноса, поворота и масштабирования (может быть описано матрицей).

### PCA для уменьшения размерности

Использование PCA для уменьшения размерности подразумевает обнуление одного или нескольких наименьших главных компонент, что приводит к проекции данных, которая сохраняет максимальную дисперсию.

In [ ]:
pca = PCA(n_components=1)
pca.fit(X)
X_pca = pca.transform(X)
print("Исходный размер:", X.shape)
print("Размер после преобразования:", X_pca.shape)

Преобразованные данные были сведены к одному измерению.

Чтобы понять эффект этого снижения размерности, мы можем выполнить обратное преобразование этих сокращенных данных и построить его вместе с исходными данными.

In [ ]:
X_new = pca.inverse_transform(X_pca)
plt.scatter(X[:, 0], X[:, 1], alpha=0.2)
plt.scatter(X_new[:, 0], X_new[:, 1], alpha=0.8)
plt.axis('equal');

Светлые точки — это исходные данные, а темные точки — их спроецированная версия.

Это проясняет, что означает уменьшение размерности PCA: информация вдоль наименее важной главной оси или осей удаляется, оставляя только компонент(ы) данных с самой высокой дисперсией.
Доля удаляемой дисперсии (пропорциональная разбросу точек относительно линии, образованной на предыдущем рисунке) является примерной мерой того, сколько «информации» отбрасывается при этом уменьшении размерности.

### PCA для визуализации: рукописные цифры

Польза снижения размерности может быть не совсем очевидна только в двух измерениях, но она становится ясной при рассмотрении многомерных данных.

Давайте рассмотрим применение PCA к набору данных цифр, с которым мы уже работали.

In [ ]:
from sklearn.datasets import load_digits
digits = load_digits()
digits.data.shape

Напомним, что набор данных digits состоит из изображений размером 8 × 8 пикселей, что означает, что они являются 64-мерными.

Чтобы получить некоторое представление о связях между этими точками, мы можем использовать PCA, чтобы спроецировать их на более управляемое количество измерений, скажем, на два.

In [ ]:
pca = PCA(2)  # project from 64 to 2 dimensions
projected = pca.fit_transform(digits.data)
print(digits.data.shape)
print(projected.shape)

In [ ]:
plt.scatter(projected[:, 0], projected[:, 1],
            c=digits.target, edgecolor='none', alpha=0.5,
            cmap=plt.cm.get_cmap('rainbow', 10))
plt.xlabel('component 1')
plt.ylabel('component 2')
plt.colorbar();

### Что означают компоненты?

Здесь мы можем пойти немного дальше и начать спрашивать, что *означают* сокращенные измерения.
Это значение можно понять в терминах комбинаций базисных векторов.
Например, каждое изображение в обучающем наборе определяется набором из 64 значений пикселей, которые мы назовем вектором $x$:

$$
x = [x_1, x_2, x_3 \cdots x_{64}]
$$

Один из способов думать об этом — в терминах базиса пикселей.
То есть, чтобы построить изображение, мы умножаем каждый элемент вектора на пиксель, который он описывает, а затем складываем результаты вместе, чтобы построить изображение:

$$
{\rm image}(x) = x_1 \cdot{\rm (pixel~1)} + x_2 \cdot{\rm (pixel~2)} + x_3 \cdot{\rm (pixel~3)} \cdots x_{64} \cdot{\rm (pixel~64)}
$$

Один из способов, который мы могли бы себе представить, чтобы уменьшить размерность этих данных, — это обнулить все, кроме нескольких из этих базисных векторов.

Но попиксельное представление — это не единственный выбор базиса. Мы также можем использовать другие базисные функции, каждая из которых содержит некоторый предопределенный вклад от каждого пикселя, и записать что-то вроде:

$$
{\rm image}(x) = {\rm mean} + x_1 \cdot{\rm (basis~1)} + x_2 \cdot{\rm (basis~2)} + x_3 \cdot{\rm (basis~3)} \cdots
$$

PCA можно рассматривать как процесс выбора оптимальных базисных функций, так что сложения только первых нескольких из них достаточно для подходящей реконструкции основной массы элементов в наборе данных.
Главные компоненты, которые действуют как низкоразмерное представление наших данных, являются просто коэффициентами, которые умножают каждый из элементов в этой серии.
На следующем рисунке показано похожее изображение реконструкции той же цифры с использованием среднего значения плюс первые восемь базисных функций PCA.

### Выбор количества компонентов

Важнейшей частью использования PCA на практике является возможность оценить, сколько компонентов необходимо для описания данных.
Это можно определить, посмотрев на кумулятивное *объясненное отношение дисперсии* как функцию количества компонент.

In [ ]:
pca = PCA().fit(digits.data)
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance');

Эта кривая количественно определяет, какая часть общей 64-мерной дисперсии содержится в первых $N$ компонентах.
Например, мы видим, что с цифровыми данными первые 10 компонентов содержат приблизительно 75% дисперсии, в то время как вам нужно около 50 компонентов, чтобы описать почти 100% дисперсии.

Это говорит нам о том, что наша 2-мерная проекция теряет много информации (измеряемой по объясненной дисперсии) и что нам понадобится около 20 компонентов, чтобы сохранить 90% дисперсии. Рассмотрение этого графика для многомерного набора данных может помочь вам понять уровень избыточности, присутствующий в его признаках.

## PCA для фильтрации шума

PCA также можно использовать для фильтрации зашумленных данных.
Идея заключается в следующем: любые компоненты с дисперсией, намного большей, чем влияние шума, должны быть относительно незатронуты шумом.
Таким образом, если вы реконструируете данные, используя только самое большое подмножество главных компонентов, вы должны предпочтительно сохранить сигнал и отбросить шум.

Давайте посмотрим, как это выглядит с цифровыми данными.

In [ ]:
def plot_digits(data):
    fig, axes = plt.subplots(4, 10, figsize=(10, 4),
                             subplot_kw={'xticks':[], 'yticks':[]},
                             gridspec_kw=dict(hspace=0.1, wspace=0.1))
    for i, ax in enumerate(axes.flat):
        ax.imshow(data[i].reshape(8, 8),
                  cmap='binary', interpolation='nearest',
                  clim=(0, 16))
plot_digits(digits.data)

Теперь давайте добавим немного случайного шума, чтобы создать зашумленный набор данных, и перерисуем.

In [ ]:
rng = np.random.default_rng(42)
rng.normal(10, 2)

In [ ]:
rng = np.random.default_rng(42)
noisy = rng.normal(digits.data, 4)
plot_digits(noisy)

Визуализация наглядно демонстрирует наличие этого случайного шума. 

Давайте обучим модель PCA на зашумленных данных, попросив, чтобы проекция сохранила 50% дисперсии.

In [ ]:
pca = PCA(0.50).fit(noisy)
pca.n_components_

Здесь 50% дисперсии составляют 12 главных компонентов из 64 исходных признаков. 

Теперь мы вычисляем эти компоненты, а затем используем обратное преобразование для реконструкции отфильтрованных цифр.

In [ ]:
components = pca.transform(noisy)
filtered = pca.inverse_transform(components)
plot_digits(filtered)

Это свойство сохранения сигнала/фильтрации шума делает PCA очень полезной процедурой выбора признаков — например, вместо того, чтобы обучать классификатор на очень многомерных данных, вы можете вместо этого обучать классификатор на представлении главного компонента меньшей размерности, что автоматически будет отфильтровывать случайный шум во входных данных.

## Пример: Eigenfaces

In [ ]:
from sklearn.datasets import fetch_lfw_people
faces = fetch_lfw_people(min_faces_per_person=60)
print(faces.target_names)
print(faces.images.shape)

Давайте рассмотрим главные оси, которые охватывают этот набор данных.
Поскольку это большой набор данных, мы будем использовать "случайный" svd_solver в `PCA`, он использует рандомизированный метод для более быстрого приближения первых $N$ главных компонентов, чем стандартный подход, за счет некоторой потери точности. Этот компромисс может быть полезен для высокоразмерных данных (здесь размерность почти 3000).

In [ ]:
pca = PCA(150, svd_solver='randomized', random_state=42)
pca.fit(faces.data)

В этом случае может быть интересно визуализировать изображения, связанные с первыми несколькими главными компонентами (эти компоненты технически известны как *собственные векторы*, поэтому такие типы изображений часто называют *собственными лицами*):

In [ ]:
fig, axes = plt.subplots(3, 8, figsize=(9, 4),
                         subplot_kw={'xticks':[], 'yticks':[]},
                         gridspec_kw=dict(hspace=0.1, wspace=0.1))
for i, ax in enumerate(axes.flat):
    ax.imshow(pca.components_[i].reshape(62, 47), cmap='bone')

Результаты очень интересны и дают нам представление о том, как изменяются изображения: например, первые несколько собственных лиц (сверху слева), похоже, связаны с углом освещения лица, а последующие главные векторы, похоже, выделяют определенные черты, такие как глаза, носы и губы.

In [ ]:
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance');

150 выбранных нами компонентов составляют чуть более 90% дисперсии.

Это привело бы нас к мысли, что, используя эти 150 компонентов, мы восстановим большинство основных характеристик данных.

Чтобы сделать это более конкретным, мы можем сравнить входные изображения с изображениями, восстановленными из этих 150 компонентов.

In [25]:
# Compute the components and projected faces
pca = pca.fit(faces.data)
components = pca.transform(faces.data)
projected = pca.inverse_transform(components)

In [ ]:
# Plot the results
fig, ax = plt.subplots(2, 10, figsize=(10, 2.5),
                       subplot_kw={'xticks':[], 'yticks':[]},
                       gridspec_kw=dict(hspace=0.1, wspace=0.1))
for i in range(10):
    ax[0, i].imshow(faces.data[i].reshape(62, 47), cmap='binary_r')
    ax[1, i].imshow(projected[i].reshape(62, 47), cmap='binary_r')

ax[0, 0].set_ylabel('full-dim\ninput')
ax[1, 0].set_ylabel('150-dim\nreconstruction');

Эта визуализация ясно показывает, почему выбор признаков PCA используют: он уменьшает размерность данных почти в 20 раз, при этом проецируемые изображения содержат достаточно информации, чтобы мы могли на глаз распознать людей на каждом изображении. 